# Project 6
Seth Beckett & Carson Stoker

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

sns.set_theme()

## Data Exploration

In [ ]:
hydro = pd.read_csv("data/RRCA_baseflow.csv")
hydro.head()

In [ ]:
# fix date variable, add days var representing days since 1900-01-01, and add xy variable
hydro['days'] = hydro.Date - 693963
hydro['Date'] = pd.to_datetime(hydro['days'] ,origin='1900-01-01', unit='D')
hydro['xy'] = hydro.x.astype(str) + ', ' + hydro.y.astype(str)
hydro.head()

In [ ]:
# look at target variable over time, split by segment_id
sns.scatterplot(hydro, x='Date', y='Observed', hue='Segment_id')
plt.show()

In [ ]:
# look at outliers
plt.boxplot(hydro["Observed"], vert=False)
plt.xlabel('Observed')
plt.show()

In [ ]:
# higher segment_id's correlate to the first chunk that causes the skew?
sns.scatterplot(hydro, x='Segment_id', y='Observed')

In [ ]:
# find what the two outlier groups are
hydro.loc[hydro.Segment_id > 210].Segment_id.unique()

In [ ]:
# look at range of dates for those two segments (seem to be old and not relevant)
hydro.loc[(hydro.Segment_id == 239) | (hydro.Segment_id == 256)].Date.describe()

In [ ]:
# look at outliers
plt.boxplot(hydro.loc[hydro.Segment_id < 220].Observed, vert=False)
plt.xlabel('Observed')
plt.show()

In [ ]:
# look at baseflow over time when bad segs removed
hydro_2 = hydro.loc[hydro.Segment_id < 220]
sns.scatterplot(hydro_2, x='Date', y='Observed', hue='Segment_id')
plt.show()


In [ ]:
# check out explanatory variables in comparison to response variable when segments 239 and 256 aren't removed
fig, axs = plt.subplots(3, 2)
sns.scatterplot(data=hydro, x='x', y='Observed', ax=axs[0,0], alpha=0.3, color='black')
sns.scatterplot(data=hydro, x='y', y='Observed', ax=axs[1,0], alpha=0.3, color='black')
sns.scatterplot(data=hydro, x='Evapotranspiration', y='Observed', ax=axs[2,0], alpha=0.3, color='black')
sns.scatterplot(data=hydro, x='Precipitation', y='Observed', ax=axs[0,1], alpha=0.3, color='black')
sns.scatterplot(data=hydro, x='Irrigation_pumping', y='Observed', ax=axs[1,1], alpha=0.3, color='black')


plt.tight_layout()
plt.show()


In [ ]:
# check out vars when 239 and 256 are removed
fig, axs = plt.subplots(3, 2)
sns.scatterplot(data=hydro_2, x='x', y='Observed', ax=axs[0,0], alpha=0.3, color='black')
sns.scatterplot(data=hydro_2, x='y', y='Observed', ax=axs[1,0], alpha=0.3, color='black')
sns.scatterplot(data=hydro_2, x='Evapotranspiration', y='Observed', ax=axs[2,0], alpha=0.3, color='black')
sns.scatterplot(data=hydro_2, x='Precipitation', y='Observed', ax=axs[0,1], alpha=0.3, color='black')
sns.scatterplot(data=hydro_2, x='Irrigation_pumping', y='Observed', ax=axs[1,1], alpha=0.3, color='black')


plt.tight_layout()
plt.show()

## Model Fitting

In [ ]:
# model with all data
X = hydro_2.drop(columns=['Date', 'Observed'])
X = pd.get_dummies(X, columns=['Segment_id', 'xy'], drop_first=True)
y = hydro_2['Observed']

results = sm.OLS(y, sm.add_constant(X)).fit()
print(results.summary())

In [ ]:
# make sure vars are all significant
results.pvalues.sort_values(ascending=False).head()

All p-values are significant, but it might make it a little cleaner and more interpretable to cut out variables that might be highly collinear, like the segments and xy.

In [ ]:
# model without x, y, or segments (FINAL MODEL)
X = hydro_2.drop(columns=['Date', 'Observed', 'Segment_id', 'x', 'y', 'days'])
X = pd.get_dummies(X, columns=['xy'], drop_first=True)
y = hydro_2['Observed']

results = sm.OLS(y, sm.add_constant(X)).fit()
print(results.summary())

In [ ]:
# make sure all vars are significant
results.pvalues.sort_values(ascending=False).head()

In [ ]:
# check out coefficients
results.params.head()

In [ ]:
# check out most extreme coefficients
print(results.params.sort_values(ascending=False).head(10))
print()
print(results.params.sort_values(ascending=True).head(10))

In [ ]:
# look at info on each of the quantitative variables
quant_vars = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping', 'days']
hydro_2[quant_vars].describe()

Below I used this model to test against the above, but our above gets a better r^2 and is just as explainable

In [ ]:
# model without x y or xy (segments might be better for explainability)
X = hydro_2.drop(columns=['Date', 'Observed', 'xy', 'x', 'y'])
X = pd.get_dummies(X, columns=['Segment_id'], drop_first=True)
y = hydro_2['Observed']

results = sm.OLS(y, sm.add_constant(X)).fit()
print(results.summary())

Because our model using stations dummied out gets such a good adjusted r2 and the variables are explainable, we'll use that!

# RANDOM WORK BELOW HERE
Below here is a bunch of work we did before realizing that our results were going to be super weird and not as interpretable if we included the data from segments 239 and 256, because they accounted for all of the baseflow above 220ish, and observations only ran from 1939-1949.

Also we started doing a bunch of stuff outside of the scope of this project that we didn't fully understand (time-series and engineering lag features) so we're just doing the basics here.

## 2. Initial Model
Here we will fit an initial model and see if it meets the linear model assumptions

In [ ]:
# since x and y have to do with a discrete location on the river, we will also use them as a dummy variable,
# but we'll keep the separate x and y to predict as quantitative and see if it works
hydro['xy'] = hydro.x.astype(str) + ', ' + hydro.y.astype(str)
X = pd.get_dummies(hydro, columns=["Segment_id", "xy"], drop_first=True).drop(columns=["Date", "Observed"])
X = sm.add_constant(X)
y = hydro["Observed"]
display(X.head())
display(y.head())

In [ ]:
model_0 = sm.OLS(y, X)
results_0 = model_0.fit()
print(results_0.summary())


In [ ]:
# try dropping x & y since we made them categorical
X = X.drop(columns=['x', 'y'])

model_1 = sm.OLS(y, X)
results_1 = model_1.fit()
print(results_1.summary())

In [ ]:
# check assumptions (many use the errors)
y_pred = results_1.predict(X)
errors = y - y_pred

In [ ]:
# normality of errors
sm.qqplot(errors, alpha=0.25)
plt.show()

In [ ]:
# check error vs pred
sns.scatterplot(x=y_pred, y=errors, alpha=0.3)

## 3. Fix Violated Assumptions
Now we attempt to fix the violated assumptions, and will recheck the model for violations and check it's metrics

## 4. Time-Series Feature Engineering
Here we will attempt to improve our model by looking at whether autocorrelation exists, and if it does we will create lag features.

In [ ]:
# look at autocorrelation
fig, axes = plt.subplots(2, 1, figsize=(10,8))
plot_acf(y, ax=axes[0])
plot_pacf(y, ax=axes[1])
plt.show()

This strongly suggests that the observed baseflow is highly influenced by previous measurements of observed baseflow

In [ ]:
# add season feature to account for time


## 5. Final Model and Conclusions

In [ ]:
# standardize to interpret coefficients
quant_cols = ['Evapotranspiration', 'Precipitation', 'Irrigation_pumping']
for col in quant_cols:
    X[col] = (X[col] - X[col].mean()) / X[col].std()

In [ ]:
model_3 = sm.OLS(y, X)
results_3 = model_3.fit()
print(results_3.summary())

In [ ]:
# check out how the quantitative variables affect the model
results_3.params[quant_cols].sort_values(ascending=False)

In [ ]:
# check out the most extreme coefficients of all variables
print(results_3.params.sort_values(ascending=False).head(10))
print()
print(results_3.params.sort_values().head(10))

In [ ]:
# try removing combined xy since it's literally the same as Segment_id (almost exactly)
X_new = hydro.drop(columns=['xy', 'x', 'y', 'Date', 'Observed'])
for col in quant_cols:
    X_new[col] = (X_new[col] - X_new[col].mean()) / X_new[col].std()

In [ ]:
model_4 = sm.OLS(y, sm.add_constant(X_new))
results_4 = model_4.fit()
print(results_4.summary())

In [ ]:
# terrible R^2, lets try throwing xy or x and y back in
X_new = hydro.drop(columns=['x', 'y', 'Date', 'Observed'])
for col in quant_cols:
    X_new[col] = (X_new[col] - X_new[col].mean()) / X_new[col].std()

model_5 = sm.OLS(y, sm.add_constant(pd.get_dummies(X_new, columns=['xy'], drop_first=True)))
results_5 = model_5.fit()
print(results_5.summary())

In [ ]:
# arguably best model since least vars and most r2, so check out extreme coefficients
print(results_5.params.sort_values(ascending=False).head(20))
print()
print(results_5.params.sort_values().head(20))

In [ ]:
# check if it's the same with x and y and not xy
X_new = hydro.drop(columns=['xy', 'Date', 'Observed'])

quant_cols.append('x')
quant_cols.append('y')

for col in quant_cols:
    X_new[col] = (X_new[col] - X_new[col].mean()) / X_new[col].std()

model_6 = sm.OLS(y, sm.add_constant(X_new))
results_6 = model_6.fit()
print(results_6.summary())

In [ ]:
X_new = hydro.drop(columns=['x', 'y', 'Date', 'Observed', 'xy'])
X_new = pd.get_dummies(X_new, columns=['Segment_id'], drop_first=True)
model_7 = sm.OLS(y, sm.add_constant(pd.get_dummies(X_new)))
results_7 = model_7.fit()
print(results_7.summary())

Conclusion: Using dummied out segments or dummied out xy combos are huge in getting a higher adjusted r2

In [ ]:
X_final = hydro.drop(columns=['x', 'y', 'Date', 'Observed', 'Segment_id', 'days'])
X_final = pd.get_dummies(X_final, columns=['xy'], drop_first=True)

final_model = sm.OLS(y, sm.add_constant(pd.get_dummies(X_final)))
final_results = final_model.fit()
print(final_results.summary())